# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs vroom iinput output formats:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

http://localhost/notebooks/tests/test5.ipynb 2 veicoli 1 solo mattina 2 corse

http://localhost/notebooks/tests/test6.ipynb 2 veicoli tutto il giorno 4 corse con priorita


In [5]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time
import polyline

vroom_url = 'http://vroom:3000/optimization/'
vehicleProfile = "driving-car"

# DATI posizioni
geoA = json.load(open('../data/mezzolombardo.geojson'))['features']   #4 points
geoB = json.load(open('../data/mezzocorona.geojson'))['features']     #6 points
geoC = json.load(open('../data/piana_rotaliana.geojson'))['features'] #6 points distanti

pointsA = [ p['geometry']['coordinates'] for p in geoA ]
pointsB = [ p['geometry']['coordinates'] for p in geoB ]
pointsC = [ p['geometry']['coordinates'] for p in geoC ]

#drt_example = json.load(open('../data/drt_2_veicoli_4_corse.json'))
drt_example = json.load(open('../data/drt_2_veicoli-fulltime_4_corse-priorita.json'))

for v in drt_example['vehicles']:
    v['profile'] = vehicleProfile

zoom = 14
#mapBbox = [11.077569,46.187199,11.161166,46.230027] #grande
#mapBbox = [11.093445,46.212002,11.112542,46.220049]  #piccolo

point_start =  [11.10249,46.21579] #posizione iniziale veicoli
C = [
      [7.425727844238279,44.29436701558007],
      [7.666740417480469,44.29436701558007],
      [7.666740417480469,44.47274618357197],
      [7.425727844238279,44.47274618357197],
      [7.425727844238279,44.29436701558007]
    ]
##Cuneo polygon coordinates

Cstart =  [7.574901580810547,44.407910474534425]
mapBbox = [C[0][0],C[0][1],C[1][0],C[2][1]]

point_start = Cstart.copy()


center = point_start.copy()
point_end = point_start.copy()

def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]

def randomPoint(bb):
    min_x, min_y, max_x, max_y = bb
    x = random.uniform(min_x, max_x)
    y = random.uniform(min_y, max_y)
    return [x, y]

def randomColor():
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r(),r(),r())

def vehicle2Map(v):
    idv = str(v['id'])
    cap = str(v['capacity'][0])

    try:
        loc = v['start']
        times = v['time_window']
    except:
        loc = None
        times = []
    
    mText = "VEICOLO %s" % idv
    #mText += "<br> capacity %s" % cap
    
    if len(times) > 0:
        mText += "<br>Orario turno <br> da %s a %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = randomColor()
    
    if loc is not None:
        mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right',permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
        mark.add_to(Map)

def job2Map(j):
    idj = str(j['id'])    
    loc = j['location']
    if hasattr(j,'time_windows'):
        times = j['time_windows'][0]
    else:
        times = []
    
    mText = "job %s<br>" % (idj)
    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "da: %s <br> a: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )

    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right', permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

def ship2Map(s):

    color = COLORS[str(s['pickup']['id'])]
    
    # prenotazione PARTENZA
    loc1 = s['pickup']['location']
    mText = "ID %s" % s['pickup']['id']
    if s['priority']>0:
        mText +=" (CONFERMATA)"
    
    try:
        times = s['pickup']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "<br>Partenza orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    #mText += "Quantita %s <br>" % s['amount'][0]
    #mText += "Priority %s <br>" % s['priority']
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True) 
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc1), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)
    
    # prenotazione ARRIVO
    loc2 = s['delivery']['location']
    mText = "ID %s <br>" % s['delivery']['id']
    try:
        times = s['delivery']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "Arrivo orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )

    #mText += "Quantita %s <br>" % s['amount'][0]
    
    folium.PolyLine(locations = [revcc(loc1),revcc(loc2)], color = color, opacity = 0.6, weight = 3).add_to(Map)
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True)
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc2), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)

def step2Map(step):
    sid = ''
    wait = 0
    try:
        loc = step['location']
        sid = step['id']
        wait = step['waiting_time']
    except:
        loc = None

    if loc and step['type'] != 'end':
        tit = str(i)+" "+step['type']+"<br> arrivo: %s " % sec2hour(step['arrival'])

        mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(tit, direction='bottom', permanent=True, style='background:yellow'), icon=folium.Icon(icon='circle', prefix='fa', color='green'))
        mark.add_to(Map)

def step2Print(step):
    sid = ''
    wait = 0
    try:
        loc = step['location']
        sid = step['id']
        wait = step['waiting_time']
    except:
        loc = None

    if sid:
        sText = "CORSA %s alle %s - %s" % ( sid, sec2hour(step['arrival']), step['type'] )
    else:
        sText = "%s alle %s " % (step['type'].upper(), sec2hour(step['arrival']) )

    if wait :
        sText += " - Attesa %s " % str(sec2hour(wait))
    
    print(sText)

def route2Map(route):
    geomdecoded = ors.convert.decode_polyline(route['geometry'])
    folium.PolyLine(
        locations = [revcc(coords) for coords in geomdecoded['coordinates']],
        color = rcolor,
        opacity = 0.8,
        weight = 5
    ).add_to(Map)

#CUSTOM ICONS https://fontawesome.com/icons?d=gallery&p=2

Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

COLORS = {
'1':'gray','2':'darkblue','3':'black','4':'beige','5':'lightgreen',
'6':'white','7':'darkpurple','8':'lightgray','9':'purple','10':'red',
'11':'green','12':'orange','13':'darkred','14':'pink','15':'cadetblue',
'16':'blue','17':'darkgreen','18':'lightblue','19':'lightred'
}



HOURS1 = [
[hour2sec('08:00'),hour2sec('08:05')],
[hour2sec('09:00'),hour2sec('09:05')],
[hour2sec('10:00'),hour2sec('10:05')],
[hour2sec('11:00'),hour2sec('11:05')],
[hour2sec('12:00'),hour2sec('12:05')]
]

HOURS2 = [
[hour2sec('14:00'),hour2sec('14:05')],
[hour2sec('15:00'),hour2sec('15:05')],
[hour2sec('16:00'),hour2sec('16:05')],
[hour2sec('17:00'),hour2sec('17:05')],
[hour2sec('18:00'),hour2sec('18:05')]
]
# VEICOLI

HOURSall = HOURS1 + HOURS2

vehicles = []

### VEICOLO 1
vid = 1

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [1],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,       # posizione di partenza veicoli
    'end': loc,
    'time_window': [              #orario lavoro del veicolo 1
        hour2sec('07:00'),
        hour2sec('19:00')
    ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2

loc = drt_example['vehicles'][1]['start']

vehicle = {
    'id': vid,
    'capacity': [1],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,        # posizione di partenza veicoli
    'end': loc,
    'time_window': [                  # pomeridiano
        hour2sec('07:00'),
        hour2sec('19:00')
    ],
#     'breaks': [
#         {
#           "id": 1,
#           "service": 300,
#           "time_windows": [ #an array of time_window objects describing valid slots for break start
#             [hour2sec('10:00'),hour2sec('15:00')],
#           ]
#         }
#     ]
}
vehicles.append(vehicle)

points = pointsA + pointsB

shipsProgrammed = [1,2,3]    # SIMULA ELENCO PRENOTAZIONI PROGRAMMATE( elenco id prenotazioni)

shipments = []

GENERATED_SHIPS = 10        #NUMERO SHIPMENTS GENERATE

for i in list(range(GENERATED_SHIPS)):
    
    sid = i+1
    
    ship = {          #struttura minima valida per prenotazione
        'pickup': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },
        'delivery': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },
        'priority': 0,
        'amount': [ 1 ]        
    }
    
    if sid in shipsProgrammed:
        ship['priority']= 100;

    shiptype = random.choice(['pickup','delivery']) # pickup == PARTENZA, delivery == ARRIVO
    
    ship[ shiptype ]['time_windows'] = [ random.choice(HOURSall) ]
    
    shipments.append(ship)

#vehicles = [vehicles[0]]    #solo primo veicolo

request = {
     'vehicles': vehicles,
     'shipments': shipments,
     'jobs': [],
}

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for v in request['vehicles']:
    vehicle2Map(v)

#print(shipments)

for s in request['shipments']:
    
    ship2Map(s)
    timeStart = '     '
    timeEnd = '     '
    confirmed = ''

    try:
        timeStart = sec2hour(s['pickup']['time_windows'][0][0])
    except:
        timeStart = '     '
    
    try:
        timeEnd = sec2hour(s['delivery']['time_windows'][0][0])
    except:
        timeEnd = '     '
    
    try:
        if s['priority'] > 0:
            confirmed='confermata'
    except:
        confirmed = ''
    
    shipText = "CORSA %s - partenza %s arrivo %s %s" % (s['pickup']['id'], timeStart, timeEnd, confirmed)
    print(shipText)

Map

CORSA 1 - partenza 18:00 arrivo       confermata
CORSA 2 - partenza 10:00 arrivo       confermata
CORSA 3 - partenza 16:00 arrivo       confermata
CORSA 4 - partenza 16:00 arrivo       
CORSA 5 - partenza 17:00 arrivo       
CORSA 6 - partenza       arrivo 11:00 
CORSA 7 - partenza       arrivo 14:00 
CORSA 8 - partenza 11:00 arrivo       
CORSA 9 - partenza       arrivo 12:00 
CORSA 10 - partenza 09:00 arrivo       


<ipython-input-5-0977e3f07a95>:103: UserWarning: color argument of Icon should be one of: {'gray', 'darkblue', 'black', 'beige', 'lightgreen', 'white', 'darkpurple', 'lightgray', 'purple', 'red', 'green', 'orange', 'darkred', 'pink', 'cadetblue', 'blue', 'darkgreen', 'lightblue', 'lightred'}.
  mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right',permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))


In [6]:
request['geometry'] = True
solution = requests.post(vroom_url, json = request).json()

for v in request['vehicles']:
    vehicle2Map(v)

print(len(shipments), "CORSE richieste\n")
print("CORSE programmate:", shipsProgrammed )

if len(solution['routes']) > 0:
    for route in solution['routes']:
        #rcolor = COLORS[str(route['vehicle'])]
        rcolor = randomColor()
        rtext = "distance: %s delivered quantity: %s pickup quantity: %s "  % ( toKm(route['distance']), route['delivery'][0], route['pickup'][0] )
        rtext = rtext+", durata: %s " % toMin(route['duration']+route['service'])
        print("\nVehicle "+str(route['vehicle']), rcolor)
        print(rtext)
        route2Map(route)
        for step in route['steps']:
            step2Print(step)
            step2Map(step)

    print("\nCORSE NON ASSEGNATE:")
    for un in solution['unassigned']:
        print('CORSA '+str(un['id']))
    
    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))
    
Map

10 CORSE richieste

CORSE programmate: [1, 2, 3]


<ipython-input-5-0977e3f07a95>:103: UserWarning: color argument of Icon should be one of: {'gray', 'darkblue', 'black', 'beige', 'lightgreen', 'white', 'darkpurple', 'lightgray', 'purple', 'red', 'green', 'orange', 'darkred', 'pink', 'cadetblue', 'blue', 'darkgreen', 'lightblue', 'lightred'}.
  mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right',permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))


KeyError: 'routes'

In [52]:
#for route in solution['routes']:
#    route['geometry'] = ors.convert.decode_polyline(route['geometry'])
#    print(json.dumps(route['steps'], indent=2))

print(json.dumps(solution, indent=2))


{
  "code": 0,
  "summary": {
    "cost": 13553,
    "unassigned": 4,
    "delivery": [
      18
    ],
    "amount": [
      18
    ],
    "pickup": [
      18
    ],
    "service": 0,
    "duration": 13553,
    "waiting_time": 56116,
    "priority": 600,
    "distance": 161764,
    "computing_times": {
      "loading": 29,
      "solving": 40,
      "routing": 62
    }
  },
  "unassigned": [
    {
      "id": 20,
      "location": [
        11.10257277086855,
        46.21744624245195
      ]
    },
    {
      "id": 20,
      "location": [
        11.138173710602,
        46.189441847415445
      ]
    },
    {
      "id": 13,
      "location": [
        11.11643736263891,
        46.1917702058091
      ]
    },
    {
      "id": 13,
      "location": [
        11.080008890691085,
        46.19918814360606
      ]
    }
  ],
  "routes": [
    {
      "vehicle": 1,
      "cost": 7510,
      "delivery": [
        10
      ],
      "amount": [
        10
      ],
      "pickup": [
    

In [54]:
print(json.dumps(request, indent=2))


{
  "vehicles": [
    {
      "id": 1,
      "capacity": [
        1
      ],
      "profile": "driving-car",
      "start": [
        11.106489999999999,
        46.219789999999996
      ],
      "end": [
        11.106489999999999,
        46.219789999999996
      ],
      "time_window": [
        1620543600,
        1620586800
      ]
    },
    {
      "id": 2,
      "capacity": [
        1
      ],
      "profile": "driving-car",
      "start": [
        11.098895072937012,
        46.20291369914678
      ],
      "end": [
        11.098895072937012,
        46.20291369914678
      ],
      "time_window": [
        1620543600,
        1620586800
      ]
    }
  ],
  "shipments": [
    {
      "pickup": {
        "id": 1,
        "location": [
          11.125746041109961,
          46.21365738038296
        ]
      },
      "delivery": {
        "id": 1,
        "location": [
          11.14843474809134,
          46.210508794361985
        ],
        "time_windows": [
          [